# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [10]:
import pandas as pd
import os

In [24]:
%cd source\repos\jacksonchen1998.github.io\markdown_generator

C:\Users\user\source\repos\jacksonchen1998.github.io\markdown_generator


## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [36]:
%pwd
%ls

 磁碟區 C 中的磁碟是 OS
 磁碟區序號:  2609-7724

 C:\Users\user\source\repos\jacksonchen1998.github.io\markdown_generator 的目錄

2022/09/22  上午 11:42    <DIR>          .
2022/09/22  上午 11:42    <DIR>          ..
2022/09/22  上午 11:42            13,229 publications.ipynb
2022/09/22  上午 11:42             3,995 publications.py
2022/09/22  上午 11:42               824 publications.tsv
2022/09/22  上午 11:42             8,987 PubsFromBib.ipynb
2022/09/22  上午 11:42             6,178 pubsFromBib.py
2022/09/22  上午 11:42               499 readme.md
2022/09/22  上午 11:42            14,396 talks.ipynb
2022/09/22  上午 11:42             4,111 talks.py
2022/09/22  下午 01:03             1,235 talks.tsv
               9 個檔案          53,454 位元組
               2 個目錄  236,946,997,248 位元組可用


In [22]:
%pwd

'C:\\Users\\user'

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [37]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Addressing Function Approximation Error in Act...,Talk,talk-1,NYCU-ED708,2022-09-20,"Hsinchu city, Taiwan ...",NaN,NaN
1,Tutorial 1 on Relevant Topic in Your Field,Tutorial,tutorial-1,UC-Berkeley Institute for Testing Science,2013-03-01,"Berkeley CA, USA",http://exampleurl.com,"This is a description of your tutorial, note t..."
2,Talk 2 on Relevant Topic in Your Field,Talk,talk-2,London School of Testing,2014-02-01,"London, UK",http://example2.com,"This is a description of your talk, which is a..."
3,Conference Proceeding talk 3 on Relevant Topic...,Conference proceedings talk,talk-3,Testing Institute of America 2014 Annual Confe...,2014-03-01,"Los Angeles, CA",NaN,This is a description of your conference proce...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [38]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [39]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [41]:
%ls ../_talks

無效的參數 - "_talks".


In [42]:
!cat ../_talks/2013-03-01-tutorial-1.md

'cat' 不是內部或外部命令、可執行的程式或批次檔。
